# 임베딩
### 임베딩 : 컴퓨터가 자연어를 처리할 수 있도록 자연어를 숫자나 백터 형태로 변환하는 것이다.
### 단어 임베딩 : 말뭉치에서 각각의 단어를 벡터를 변환하는 기법을 의미한다.

## 원-핫 인코딩
### 원-핫 인코딩 : 넘파이의 기능으로 인덱스가 부여된 단어를 숫자 벡터 0 또는 1로 변환하는 가장 기본적인 방법
#### ex) 오늘 날씨는 구름이 많아요 -> 오늘 : 0, 날씨 : 1, 구름 : 2 -> 0 = [1, 0, 0], 1 = [0, 1, 0], 2 = [0, 0, 1]

In [2]:
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = "오늘 날씨는 구름이 많아요."

nouns = komoran.nouns(text)
print(nouns)

# 단어 사전 구축 및 단어별 인덱스 부여
dics = {} # 빈 딕셔너리 변수 생성
for word in nouns: # 추출된 단어별로 인덱스 부여하기 위해 for문 사용
    if word not in dics.keys():# 동일한 단어에 서로 다른 원-핫 벡터값을 가지면 안 되므로 이미 저장된 것은 다시 저장하지 않음
        dics[word] = len(dics) # 단어는 key에 저장, 인덱스는 value에 저장
print(dics)

#원-핫 인코딩
nb_classes = len(dics) # 단어 사전의 크기 결정
targets = list(dics.values()) # 딕셔너리 타입으로 되어 있는 단어 사전을 리스트 형태로 변환
one_hot_targets = np.eye(nb_classes)[targets] # 원-핫 벡터를 만들기 위해 넘파이의 eye()함수 사용하여 ()크기만큼의 단위행렬을 반환 
                                              # -> eye() 함수 뒤에 붙은 [targets]를 이용해 생성된 단위행렬의 순서를 
                                              # 단어 사전의 순서에 맞게 정렬
print(one_hot_targets)

['오늘', '날씨', '구름']
{'오늘': 0, '날씨': 1, '구름': 2}
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


### 원-핫 인코딩의 경우 간단한 구현 방법에 비해 좋은 성능을 가지므로 많이 사용하지만, 단순히 단어의 순서에 의한 인덱스 값을 가지게 되므로 단어의 의미나 유사한 단어와의 관계는 나타내지 못한다. 또한 단어 사전의 크기가 커짐에 따라 메모리 낭비와 계산 복잡도가 커진다.

## 희소 표현과 분산 표현
#### 앞서 소개한 원-핫 인코딩 방식에서 단어 인덱스는 1, 이를 제외한 것은 0으로 표현되는 방식을 희소 벡터라고 한다. 이처럼 단어가 희소 벡터로 표현되는 방식을 희소 표현이라고 한다. 

### 자연어 처리를 보다 나은 성능으로 처리하기 위해서는 기본 토큰이 되는 단어의 의미와 주변 단어 간의 관계가 단어 임베딩에 표현돼야한다. 그러나 희소 표현은 적합하지 않다. 이를 해결하기 위해 각 단어 간의유사성을 잘 표현하면서도 벡터공간을 절약할 수 있는 방법인 분산 표현을 사용한다.
### 분산 표현 : 한 단어의 정보가 특정 차원에 표현되지않고 여러 차원에 분산되어 표현되는 것이다. 이것으로 인해 하나의 차원에 다양한 정보를 가지고 있다.
#### ex) RGB : 3차원 형태의 벡터로 색을 표현 -> RGB(204, 255, 204)
### 신경망에서 분산 표현을 학습하는 과정에서 모든 차원의 데이터를 고르게 밀집시키므로 밀집 표현이라고 부르기도 하며, 밀집 표현으로 만들어진 벡터를 밀집벡터라고 한다.

## Word2Vec
### Word2Vec : 신경망 기반 단어 임베딩의 대표적인 방법, 해당 단어를 밀집 벡터로 표현하며 학습을 통해 의미상 비슷한 단어들을 비슷한 벡터 공간에 위치시켜 방향성을 가지게 된다. 또한 임베딩된 벡터들 간 연산이 가능하기 때문에 단어 간 관계를 계산할 수 있다. 방향성, 계산된 관계 값들을 통해 자연어의 의미를 파악하게 된다.
### Word2Vec는 CBOW, skip-gram이 있다.
#### CBOW : 맥락이라 표현되는 주변 단어들을 이용해 타깃 단어를 예측하는 신경망 모델로 입력을 주변 단어들로 구성하고 출력을 다깃 단어로 설정해 학습된 가중치 데이터를 임베딩 벡터로 활용하므로 학습 속도가 가장 빠르다.
#### skip-gram : CBOW 모델과 반대로 하나의 타깃 단어를 이용해 주변 단어들을 예측하는 신경망 모델로 CBOW 모델에 비해 예측해야하는 맥락이 많아져 임베딩 성능이 더 우수하다.


## Word2Vec : skip-gram, 모델 학습

In [12]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

In [11]:
# 네이버 영화 리뷰 데이터를 읽어 리스트로 반환하기 위한 메소드 생성
def read_review_data(filename) : 
    with open(filename, 'r', encoding='UTF8') as f :
        data = [line.split('\t') for line in f.read().splitlines()] # 간격은 탭으로 한다.
        data = data[1:] # 1번 열부터 읽으므로 파일 헤더는 제거된다.
    return data # 읽고 처리한 데이터를 return

# 학습 시간 측정 시작
start = time.time()

# 리뷰 파일 읽어오기(txt파일)
print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('./ratings.txt')
print(len(review_data))
print('1) 말뭉치 데이터 읽기 완료 : ', time.time() - start)

# 문장 단위로 명사만 추출해 학습 입력 데이터로 만듦
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료 : ', time.time() - start)

# Word2Vec 모델 학습
print('3) Word2Vec 모델 학습 시작')
model = Word2Vec(sentences=docs, # Word2Vec 모델 학습에 필요한 문장 데이터, Word2Vec 모델의 입력값으로 사용된다.
                 size=200, # 단어 임베딩 벡터의 차원(=크기)
                 window=4, # 주변 단어의 윈도우 크기(윈도우 크기란? 중심 단어를 예측하기 위해 앞, 뒤로 몇 개의 단어를 볼지 결정한 값)
                 hs=1, # 1:모델 학습에 softmax 사용
                 min_count=2, # 단어 최소 빈도 수 제한(설정된 miin_count 빈도 수 이하의 단어들은 학습하지 않는다.)
                 sg=1) # 0:CBOW 모델, 1:skip-gram 모델
print('3) Word2Vec 모델 학습 종료 : ', time.time() - start)

# 학습 모델 저장
print('4) 학습된 모델 저장')
model.save('nvmc.model')
print('4) 학습된 모델 저장 완료 : ', time.time() - start)

# 학습된 말뭉치 수, 코퍼스 내 전체 단어 수
print("corpus_count : ", model.corpus_count)
print("corpus_total_words : ", model.corpus_total_words)

1) 말뭉치 데이터 읽기 시작
200000
1) 말뭉치 데이터 읽기 완료 :  2.3273096084594727
2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료 :  98.25415635108948
3) Word2Vec 모델 학습 시작
3) Word2Vec 모델 학습 종료 :  116.79737877845764
4) 학습된 모델 저장
4) 학습된 모델 저장 완료 :  117.62719011306763
corpus_count :  200000
corpus_total_words :  1076896


### read_review_data 메소드에서 간격을 탭으로 한 이유는 읽어올 ratings.txt가 라인마다 tap을 사용해서 컬럼을 구분했기 때문이다. 따라서 각 라인별로 데이터를 읽어오기 위해서는 tap으로 구분해서 읽어와야하므로 tap으로 설정했고 컬럼을 제외하고 읽어오기 위해 헤더를 제거하였다.

## Word2Vec : skip-gram, 모델 활용

In [13]:
from gensim.models import Word2Vec

In [20]:
# 저장한 모델 불러오기
model = Word2Vec.load('nvmc.model')
print('corpus_total_words: ', model.corpus_total_words)

# '사랑'이란 단어로 생성한 단어 임베딩 벡터
print('사랑 : ', model.wv['사랑'])

# 단어 유사도 계산
print('일요일 = 월요일\t', model.wv.similarity(w1='일요일', w2='월요일'))
print('안성기 = 배우\t', model.wv.similarity(w1='안성기', w2='배우'))
print('대기업 = 삼성\t', model.wv.similarity(w1='대기업', w2='삼성'))
print('일요일 != 삼성\t', model.wv.similarity(w1='일요일', w2='삼성'))
print('히어로 != 삼성\t', model.wv.similarity(w1='히어로', w2='삼성'))

# 가장 유사한 단어 추출
print(model.wv.most_similar('안성기', topn=5))
print(model.wv.most_similar('시리즈', topn=5))

corpus_total_words:  1076896
사랑 :  [-3.88622344e-01 -1.94815725e-01  1.46532044e-01 -1.22451395e-01
  4.19196710e-02  2.67774146e-02  6.05018027e-02  8.64254683e-02
  2.99633294e-01  6.37767185e-03  1.31219789e-01 -3.98649842e-01
  1.53420806e-01  2.33451009e-01  3.25740844e-01  2.03807965e-01
 -6.83370680e-02 -2.08289355e-01 -1.29048631e-01 -1.04448674e-02
 -2.36133888e-01 -2.75666565e-01  7.40562305e-02  4.63346422e-01
  9.53263324e-03 -5.35712251e-03  3.42081077e-02 -2.64026672e-01
 -9.34771821e-03 -1.24441244e-01  4.75897104e-01 -3.60944048e-02
 -2.03737512e-01  3.36395711e-01  2.68643320e-01 -1.30586430e-01
 -3.71852189e-01 -1.37713924e-03 -3.58053505e-01 -1.50014713e-01
 -1.03704959e-01  1.64361641e-01  1.85239479e-01 -6.12507239e-02
  3.41654897e-01  1.70152292e-01 -1.05282538e-01 -2.33687386e-02
 -3.31414759e-01  2.18372807e-01 -7.63585865e-02 -2.89821833e-01
  2.10347518e-01  1.43579654e-02 -1.47121564e-01 -7.45346323e-02
  1.35818170e-02 -3.01273614e-02 -2.73640782e-01 -2.457

### model.wv.similarity(a, b) : a, b 두 단어 간의 유사도를 계산해준다. 0~1사이의 값을 가진다.
### model.wv.similarity(a, topn=n) : a 단어와 가장 유사한 단어를 리스트 형태로 반환해준다. topn에서 설정한 n 값은 반호나되는 유사한 단어 수를 의미하며 예제에서는 n=5이므로 5개의 유사한 단어를 반환했다.